# 作業目標:

    通過建立多層的神經網路, 了解權值矩陣更新

# 作業重點: 

3  層神經網路

通過增加更多的中間層，以對更多關係的組合進行建模

syn1 權值矩陣將隱層的組合輸出映射到最終結果，

而在更新 syn1 的同時，還需要更新 syn0 權值矩陣，

以從輸入資料中更好地產生這些組合

In [6]:
import numpy as np
 
# Sigmoid 函數可以將任何值都映射到一個位於 0 到  1 範圍內的值。通過它，我們可以將實數轉化為概率值
def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

X = np.array([  [0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1] ])  
#        
# define y for output dataset 
#
y = np.array([[0.1, 0.3, 0.5, 0.7]]).T
y.shape, y

((4, 1),
 array([[0.1],
        [0.3],
        [0.5],
        [0.7]]))

In [2]:

# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(1)
#亂數設定產生種子得到的權重初始化集仍是隨機分佈的，
#但每次開始訓練時，得到的權重初始集分佈都是完全一致的。
 
#    
# initialize weights randomly with mean 0
#
syn0 = 2*np.random.random((3,5)) - 1

#
# define syn1
#
syn1 = 2*np.random.random((5,1)) - 1

iter = 0
#該神經網路權重矩陣的初始化操作。
#用 “syn0” 來代指 (即“輸入層-第一層隱層”間權重矩陣）
#用 “syn1” 來代指 (即“輸入層-第二層隱層”間權重矩陣）

print(syn0, '\n', syn1)

[[-0.16595599  0.44064899 -0.99977125 -0.39533485 -0.70648822]
 [-0.81532281 -0.62747958 -0.30887855 -0.20646505  0.07763347]
 [-0.16161097  0.370439   -0.5910955   0.75623487 -0.94522481]] 
 [[ 0.34093502]
 [-0.1653904 ]
 [ 0.11737966]
 [-0.71922612]
 [-0.60379702]]


神經網路訓練
for 迴圈反覆運算式地多次執行訓練代碼，使得我們的網路能更好地擬合訓練集

In [3]:
for iter in range(10000):
    # forward propagation
    l0 = X
    l1 = nonlin(np.dot(l0,syn0))
    
    '''
    新增
    l2_error 該值說明了神經網路預測時“丟失”的數目。
    l2_delta 該值為經確信度加權後的神經網路的誤差，除了確信誤差很小時，它近似等於預測誤差。
    '''
    
    '''
              l0     w0            l1 (5 neurons)         w1            l2 (1 neuron)
              -----  ------      --------------------     ------       ---------------------
              in     syn0               act               syn1                act      out
    forward   (4x3)  (3x5)   =>  (4x5)  sigmoid (4x5)     (5x1)   =>   (4x1)  sigmoid  (4x1)
    backward                                              dw1     <=   d_l2            l2_err   
                                                                  l1.T
                     dw0     <=  d_l1           l1_err <=              d_l2
                             l0.T                      syn1.T
    '''
    # add layer2
    l2 = nonlin(np.dot(l1, syn1))                   # (4x5)(5x1) => (4x1) forward
    # how much did we miss?    
    l2_error = y - l2                               # (4x1)
    # multiply how much we missed by the 
    # slope of the sigmoid at the values in l2    
    l2_delta = l2_error * nonlin(l2, True)          # (4x1)
    # syn1 update weights
    dw1 = np.dot(l1.T, l2_delta)                  # (5x4)(4x1) => (5x1) backward
    syn1 += dw1

    # how much did we miss?
    l1_error = np.dot(l2_delta, syn1.T)             # (4x1)(1x5) => (4x5)
    # multiply how much we missed by the 
    # slope of the sigmoid at the values in l1
    l1_delta = l1_error * nonlin(l1,True)           # (4x5)
    # syn0 update weights
    dw0 = np.dot(l0.T,l1_delta)                   # (3x4)(4x5) => (3x5)
    syn0 += dw0
    
print("Output After Training:")
print("l1 output:")
print(l1)
print("l2 output:")
print(l2)
print("syn0 weights:")
print(syn0)
print("syn1 weights:")
print(syn1)

Output After Training:
l1 output:
[[0.51114329 0.57275427 0.4277172  0.84082215 0.38627483]
 [0.53672228 0.60070311 0.31025167 0.55752452 0.30935191]
 [0.6868807  0.83624515 0.19134034 0.60895943 0.14807581]
 [0.70850822 0.85142853 0.12465193 0.27085356 0.11007983]]
l2 output:
[[0.19999984]
 [0.4000001 ]
 [0.60000014]
 [0.79999977]]
syn0 weights:
[[ 0.74099579  1.33745385 -1.15015334 -1.22141774 -1.28677503]
 [ 0.10257354  0.11529811 -0.50777195 -1.43323663 -0.3401528 ]
 [ 0.04458055  0.29309747 -0.29117105  1.66435808 -0.46299812]]
syn1 weights:
[[ 0.9999082 ]
 [ 1.88364582]
 [-0.97390036]
 [-2.40357174]
 [-1.39467439]]
